In [11]:
from sqlalchemy import create_engine, Column, Integer, Text, ForeignKey, Date, Boolean
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.declarative import declarative_base
import os
# from os import path
# DB_PATH = path.split(__file__)[0] + '/test.db'

Base = declarative_base()
engine = create_engine(f'sqlite:///data.db')
ss = sessionmaker(bind=engine)()

In [13]:
class Image(Base):
    __tablename__ = 'image'
    id = Column(Integer, primary_key=True)
    name = Column(Text, unique=True)
    rtags = relationship('Tag', secondary='images2tags', backref='Images')
    star = Column(Integer, default=0)
    count = Column(Integer, default=0)
    file_url = Column(Text, default='')
    history_id = Column(Integer, ForeignKey('history.id'), default=0)
    history = relationship('History', backref='images')
    tags = Column(Text, default='')
    last_update_date = Column(Date)
    author = Column(Text)
    creator_id = Column(Integer)
    status = Column(Integer)


class Tag(Base):
    __tablename__ = 'tag'
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(Text, unique=True)


class Images2Tags(Base):
    __tablename__ = 'images2tags'
    id = Column(Integer, primary_key=True, autoincrement=True)
    game_id = Column(Integer, ForeignKey('image.id'))
    tag_id = Column(Integer, ForeignKey('tag.id'))


class History(Base):
    __tablename__ = 'history'
    id = Column(Integer, primary_key=True, autoincrement=True)
    date = Column(Date)
    start = Column(Integer)
    end = Column(Integer)
    amount = Column(Integer)
    finish = Column(Boolean, default=False)
    img_star = Column(Integer)
Base.metadata.create_all(engine)

C:\Users\20577\AppData\Local\Temp\ipykernel_11460\1478744355.py:1: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Image, and will be replaced in the string-lookup table.
  class Image(Base):


InvalidRequestError: Table 'image' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [17]:
img = Image(id=2)
ss.add(img)
ss.commit()
ss.close()

In [20]:
# Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

In [4]:
from settings import *
failed = []
from os import path
for img in ss.query(Image).filter(Image.star >= 0).all():
    if not path.exists(path.join(IMG_PATH, img.name)):
        failed.append(img)
        print(img.id)

In [30]:
from sqlalchemy import func
r = ss.query(Image).order_by(func.random()).first()
r.id

OperationalError: (sqlite3.OperationalError) no such function: rand
[SQL: SELECT image.id AS image_id, image.name AS image_name, image.star AS image_star, image.count AS image_count, image.file_url AS image_file_url, image.history_id AS image_history_id, image.tags AS image_tags, image.last_update_date AS image_last_update_date, image.author AS image_author, image.creator_id AS image_creator_id 
FROM image ORDER BY rand()
 LIMIT ? OFFSET ?]
[parameters: (1, 0)]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [2]:
from settings import *

In [3]:
imgs = ss.query(Image).filter(Image.star == -1).all()

In [4]:
import os
for img in imgs:
    if os.path.exists(IMG_PATH + '/' + img.name):
        input(str(img.id))
        os.remove(IMG_PATH + '/' + img.name)
    img.star = 0
    img.status = STATUS.DELETED

In [5]:
ss.commit()

In [12]:
ss.query(Image).update({'status': STATUS.DELETED})

129439

In [13]:
ss.commit()

In [16]:
from utils import check_exists

In [22]:
imgs = os.listdir(IMG_PATH)
for img in imgs:
    check_img = check_exists(Image, name=img)
    if check_img:
        check_img.status = STATUS.EXISTS
    else:
        if not img.startswith('yande'):
            res = input(img)
            if res:
                break
        os.remove(IMG_PATH + '/' + img)

In [23]:
ss.commit()








In [24]:
err = ss.query(Image).filter(Image.status == STATUS.DELETED, Image.star > 0)

In [26]:
for img in err.all():
    img.status = STATUS.QUEUING

In [27]:
ss.commit()